# 🎓 Personalized Education Assistant

## 📚 Learning Objectives

In this notebook, you will learn how to:
- **Create a sequential multi-agent system** using CrewAI's Process Sequential
- **Define Pydantic models** for structured outputs (learning materials, quizzes, projects)
- **Build custom tools** for generating personalized project suggestions
- **Design specialized agents** for different educational tasks
- **Coordinate tasks** that flow from one agent to another in sequence

## 🎯 Project Overview

This notebook demonstrates building an **AI-powered Personalized Education Assistant** using CrewAI. The system consists of:

1. **Learning Material Agent**: Curates videos, articles, and exercises based on user topics
2. **Quiz Creator Agent**: Generates personalized quizzes to assess understanding
3. **Project Idea Agent**: Recommends practical projects aligned with expertise level

## 🔑 Key Concepts

| Concept | Description |
|---------|-------------|
| **Process Sequential** | Tasks are executed in order, each building on the previous |
| **Structured Outputs** | Use Pydantic models to define the shape of agent outputs |
| **Custom Tools** | Build domain-specific tools for agents to use |
| **Content Curation** | Intelligently select learning materials for any topic |
| **Adaptive Learning** | Adjust recommendations based on user expertise level |

## 🔧 Requirements

- **SERPER_API_KEY**: For fetching search results via Serper API ([Get it here](https://serper.dev/))
- **OPENAI_API_KEY**: For accessing OpenAI models ([Get it here](https://platform.openai.com/api-keys))

## 🚀 Let's Get Started!

In [8]:
# ================================================================================
# STEP 1: Setup and Imports
# ================================================================================
# Import all required libraries and set up the environment

# Core CrewAI imports
from crewai import Agent, Crew, Task, Process, LLM
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool

# Pydantic for structured outputs
from pydantic import BaseModel, Field
from typing import List, Type, Optional

# Environment and utilities
import os
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# Verify API keys are set
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️ Warning: OPENAI_API_KEY not found in environment")
if not os.getenv('SERPER_API_KEY'):
    print("⚠️ Warning: SERPER_API_KEY not found in environment")

print("✅ Libraries imported successfully!")

# Initialize the LLM
llm = LLM(model="gpt-4")
print("✅ LLM initialized (GPT-4)")

✅ Libraries imported successfully!
✅ LLM initialized (GPT-4)


In [9]:
# ================================================================================
# STEP 2: Define Pydantic Models for Structured Outputs
# ================================================================================
# We'll create structured schemas to ensure consistent, well-formatted outputs

# ----------------------
# Learning Material Models
# ----------------------
class LearningResource(BaseModel):
    """A single learning resource (video, article, or exercise)"""
    title: str = Field(description="Title of the learning resource")
    resource_type: str = Field(description="Type: 'video', 'article', or 'exercise'")
    url: Optional[str] = Field(default=None, description="URL or link to the resource")
    description: str = Field(description="Brief description of what the resource covers")
    difficulty: str = Field(description="Difficulty level: 'beginner', 'intermediate', or 'advanced'")
    estimated_time: str = Field(description="Estimated time to complete (e.g., '30 minutes')")


class LearningMaterials(BaseModel):
    """Collection of curated learning materials for a topic"""
    topic: str = Field(description="The main topic of study")
    learning_path: str = Field(description="Suggested learning path or order")
    resources: List[LearningResource] = Field(description="List of learning resources")
    total_estimated_time: str = Field(description="Total estimated time to complete all resources")
    prerequisites: List[str] = Field(description="Prerequisites needed before starting")


# ----------------------
# Quiz Models
# ----------------------
class QuizQuestion(BaseModel):
    """A single quiz question"""
    question: str = Field(description="The quiz question")
    options: List[str] = Field(description="Multiple choice options (A, B, C, D)")
    correct_answer: str = Field(description="The correct answer letter (A, B, C, or D)")
    explanation: str = Field(description="Explanation of why this answer is correct")
    difficulty: str = Field(description="Question difficulty: 'easy', 'medium', or 'hard'")


class Quiz(BaseModel):
    """A complete quiz for a topic"""
    topic: str = Field(description="The topic this quiz covers")
    title: str = Field(description="Quiz title")
    instructions: str = Field(description="Instructions for taking the quiz")
    questions: List[QuizQuestion] = Field(description="List of quiz questions")
    passing_score: int = Field(description="Minimum score to pass (percentage)")
    time_limit: str = Field(description="Suggested time limit for the quiz")


# ----------------------
# Project Idea Models
# ----------------------
class ProjectIdea(BaseModel):
    """A single project idea"""
    title: str = Field(description="Project title")
    description: str = Field(description="Detailed project description")
    difficulty: str = Field(description="Project difficulty: 'beginner', 'intermediate', or 'advanced'")
    skills_practiced: List[str] = Field(description="Skills that will be practiced")
    estimated_duration: str = Field(description="Estimated time to complete")
    milestones: List[str] = Field(description="Key milestones or phases")
    resources_needed: List[str] = Field(description="Tools, libraries, or resources needed")


class ProjectSuggestions(BaseModel):
    """Collection of project suggestions"""
    expertise_level: str = Field(description="User's expertise level")
    topics: List[str] = Field(description="Topics covered by these projects")
    projects: List[ProjectIdea] = Field(description="List of project ideas")
    recommended_order: List[str] = Field(description="Recommended order to tackle projects")


print("✅ Pydantic models defined successfully!")


✅ Pydantic models defined successfully!


In [10]:
# ================================================================================
# STEP 3: Create Custom Project Suggestion Tool
# ================================================================================
# Build a custom tool that generates project ideas based on expertise level

class ProjectSuggestionInput(BaseModel):
    """Input schema for the Project Suggestion Tool"""
    topics: str = Field(description="Comma-separated list of topics of interest")
    expertise_level: str = Field(description="User's expertise level: 'beginner', 'intermediate', or 'advanced'")


class ProjectSuggestionTool(BaseTool):
    """
    A custom tool that generates tailored project ideas based on:
    - User's topics of interest
    - User's expertise level (beginner, intermediate, advanced)
    
    This tool provides a structured framework for project suggestions
    that agents can then expand upon.
    """
    name: str = "Project Suggestion Tool"
    description: str = (
        "Generates project ideas tailored to the user's expertise level and topics of interest. "
        "Input should include topics (comma-separated) and expertise_level (beginner/intermediate/advanced)."
    )
    args_schema: Type[BaseModel] = ProjectSuggestionInput

    def _run(self, topics: str, expertise_level: str) -> str:
        """
        Generate project suggestions based on expertise level and topics.
        Returns a structured JSON string with project frameworks.
        """
        topic_list = [t.strip() for t in topics.split(',')]
        level = expertise_level.lower()
        
        # Define project complexity based on expertise level
        project_frameworks = {
            "beginner": {
                "complexity": "Simple, guided projects with clear steps",
                "duration": "1-2 weeks",
                "focus": "Fundamentals and core concepts",
                "project_types": [
                    "Tutorial-based implementation",
                    "Clone a simple existing application",
                    "Create a basic tool or utility",
                    "Build a portfolio piece with guidance"
                ],
                "support_level": "High - expect to follow detailed tutorials"
            },
            "intermediate": {
                "complexity": "Multi-component projects with some autonomy",
                "duration": "2-4 weeks",
                "focus": "Integration, best practices, and problem-solving",
                "project_types": [
                    "Build a full-stack application",
                    "Create an API with database integration",
                    "Develop a data pipeline or analysis tool",
                    "Contribute to an open-source project"
                ],
                "support_level": "Medium - mix of guidance and self-direction"
            },
            "advanced": {
                "complexity": "Complex, self-directed projects with novel challenges",
                "duration": "4-8 weeks",
                "focus": "Architecture, optimization, and innovation",
                "project_types": [
                    "Design and build a distributed system",
                    "Create a library or framework for others",
                    "Implement a research paper or novel algorithm",
                    "Build a production-ready SaaS application"
                ],
                "support_level": "Low - self-directed with research required"
            }
        }
        
        framework = project_frameworks.get(level, project_frameworks["intermediate"])
        
        result = {
            "expertise_level": level,
            "topics": topic_list,
            "project_framework": framework,
            "recommendation": f"Based on {level} level expertise in {', '.join(topic_list)}, "
                             f"focus on {framework['focus']} with projects lasting {framework['duration']}."
        }
        
        return json.dumps(result, indent=2)


# Initialize the tools
project_suggestion_tool = ProjectSuggestionTool()
search_tool = SerperDevTool()

print("✅ Custom Project Suggestion Tool created!")
print("✅ SerperDev search tool initialized!")


✅ Custom Project Suggestion Tool created!
✅ SerperDev search tool initialized!


In [11]:
# ================================================================================
# STEP 4: Configure User Inputs
# ================================================================================
# Set up the user's learning preferences - topics and expertise level

# Topics the user wants to learn about
user_topics = ["Machine Learning", "Python", "Data Visualization"]

# User's current expertise level: "beginner", "intermediate", or "advanced"
expertise_level = "intermediate"

# Number of resources/questions to generate
num_resources = 5  # Learning resources to curate
num_quiz_questions = 5  # Quiz questions to create
num_projects = 3  # Project ideas to suggest

print("📚 User Configuration:")
print(f"   Topics: {', '.join(user_topics)}")
print(f"   Expertise Level: {expertise_level.capitalize()}")
print(f"   Resources: {num_resources} | Questions: {num_quiz_questions} | Projects: {num_projects}")


📚 User Configuration:
   Topics: Machine Learning, Python, Data Visualization
   Expertise Level: Intermediate
   Resources: 5 | Questions: 5 | Projects: 3


In [12]:
# ================================================================================
# STEP 5: Create the AI Agents
# ================================================================================
# Define three specialized agents for the educational pipeline

# --- Agent 1: Learning Material Curator ---
learning_material_agent = Agent(
    llm=llm,
    role="Learning Material Curator",
    goal=(
        "Curate comprehensive and high-quality learning materials "
        "tailored to the user's topics of interest and expertise level. "
        "Find the best videos, articles, and practical exercises available."
    ),
    backstory=(
        "You are an experienced educational content curator with a passion for "
        "helping learners find the best resources. You have extensive knowledge "
        "of online learning platforms, educational content creators, and academic "
        "resources. You excel at evaluating content quality, relevance, and "
        "pedagogical effectiveness. You always consider the learner's current "
        "level and create a logical learning path."
    ),
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

# --- Agent 2: Quiz Designer ---
quiz_creator_agent = Agent(
    llm=llm,
    role="Educational Quiz Designer",
    goal=(
        "Create engaging and effective quizzes that accurately assess "
        "the learner's understanding of the topics. Design questions "
        "that test both conceptual understanding and practical application."
    ),
    backstory=(
        "You are a skilled educational assessment designer with expertise in "
        "creating effective quizzes and tests. You understand learning psychology "
        "and know how to craft questions that truly measure understanding, not "
        "just memorization. You create a mix of difficulty levels to challenge "
        "learners appropriately while building their confidence. Your questions "
        "are clear, unambiguous, and provide valuable learning opportunities "
        "through detailed explanations."
    ),
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

# --- Agent 3: Project Mentor ---
project_idea_agent = Agent(
    llm=llm,
    role="Project Mentor",
    goal=(
        "Recommend practical, engaging project ideas that help learners "
        "apply their knowledge and build a portfolio. Projects should be "
        "appropriately challenging for the learner's expertise level."
    ),
    backstory=(
        "You are an experienced mentor who has guided countless learners "
        "through hands-on projects. You understand that the best learning "
        "happens through practical application. You know how to design "
        "projects that are challenging enough to promote growth but not "
        "so difficult as to cause frustration. Your project ideas are "
        "creative, relevant to industry needs, and help build impressive "
        "portfolios. You provide clear milestones and success criteria."
    ),
    tools=[project_suggestion_tool, search_tool],
    verbose=True,
    allow_delegation=False
)

print("✅ All 3 agents created successfully!")
print("   • Learning Material Curator")
print("   • Educational Quiz Designer")
print("   • Project Mentor")

✅ All 3 agents created successfully!
   • Learning Material Curator
   • Educational Quiz Designer
   • Project Mentor


In [13]:
# ================================================================================
# STEP 6: Define the Tasks
# ================================================================================
# Create three sequential tasks that build upon each other

# --- Task 1: Generate Learning Materials ---
learning_materials_task = Task(
    description=(
        f"Curate a comprehensive set of learning materials for the following topics: "
        f"{', '.join(user_topics)}.\n\n"
        f"The learner's expertise level is: {expertise_level}.\n\n"
        f"Your task:\n"
        f"1. Search for and select the {num_resources} best learning resources for each topic\n"
        f"2. Include a mix of videos, articles, and practical exercises\n"
        f"3. Ensure resources are appropriate for the {expertise_level} level\n"
        f"4. Create a logical learning path that builds knowledge progressively\n"
        f"5. Estimate time requirements for each resource\n"
        f"6. List any prerequisites needed before starting\n\n"
        f"Focus on high-quality, up-to-date resources from reputable sources."
    ),
    expected_output=(
        "A structured collection of learning materials including:\n"
        "- Topic overview and learning path\n"
        "- List of curated resources (videos, articles, exercises)\n"
        "- Each resource should have: title, type, URL, description, difficulty, estimated time\n"
        "- Total estimated time to complete\n"
        "- Prerequisites list"
    ),
    agent=learning_material_agent,
    output_json=LearningMaterials,
    output_file="learning_materials.json"
)

# --- Task 2: Create Quiz ---
quiz_creation_task = Task(
    description=(
        f"Create a comprehensive quiz to assess understanding of: {', '.join(user_topics)}.\n\n"
        f"The learner's expertise level is: {expertise_level}.\n\n"
        f"Your task:\n"
        f"1. Create {num_quiz_questions} multiple-choice questions\n"
        f"2. Include a mix of difficulty levels (easy, medium, hard)\n"
        f"3. Test both conceptual understanding and practical application\n"
        f"4. Provide 4 options (A, B, C, D) for each question\n"
        f"5. Include the correct answer and a detailed explanation\n"
        f"6. Base questions on the learning materials curated in the previous task\n\n"
        f"Ensure questions are clear, unambiguous, and educational."
    ),
    expected_output=(
        "A complete quiz including:\n"
        "- Quiz title and instructions\n"
        "- List of multiple-choice questions with:\n"
        "  - Question text\n"
        "  - Four options (A, B, C, D)\n"
        "  - Correct answer\n"
        "  - Explanation\n"
        "  - Difficulty level\n"
        "- Passing score and time limit"
    ),
    agent=quiz_creator_agent,
    output_json=Quiz,
    output_file="quiz.json",
    context=[learning_materials_task]  # Uses output from previous task
)

# --- Task 3: Suggest Projects ---
project_suggestions_task = Task(
    description=(
        f"Recommend practical project ideas for topics: {', '.join(user_topics)}.\n\n"
        f"The learner's expertise level is: {expertise_level}.\n\n"
        f"Your task:\n"
        f"1. Use the Project Suggestion Tool to get a framework for {expertise_level} level\n"
        f"2. Generate {num_projects} detailed project ideas\n"
        f"3. Ensure projects are appropriate for the {expertise_level} expertise level\n"
        f"4. Each project should apply concepts from the learning materials\n"
        f"5. Provide clear milestones and deliverables\n"
        f"6. List required tools, libraries, and resources\n"
        f"7. Suggest the recommended order to tackle the projects\n\n"
        f"Projects should be engaging, practical, and portfolio-worthy."
    ),
    expected_output=(
        "A collection of project suggestions including:\n"
        "- User's expertise level and topics\n"
        "- List of project ideas with:\n"
        "  - Title and description\n"
        "  - Difficulty level\n"
        "  - Skills practiced\n"
        "  - Estimated duration\n"
        "  - Key milestones\n"
        "  - Required resources\n"
        "- Recommended order to tackle projects"
    ),
    agent=project_idea_agent,
    output_json=ProjectSuggestions,
    output_file="project_suggestions.json",
    context=[learning_materials_task, quiz_creation_task]  # Uses output from both tasks
)

print("✅ All 3 tasks defined successfully!")
print("   • Task 1: Generate Learning Materials")
print("   • Task 2: Create Quiz (depends on Task 1)")
print("   • Task 3: Suggest Projects (depends on Tasks 1 & 2)")


✅ All 3 tasks defined successfully!
   • Task 1: Generate Learning Materials
   • Task 2: Create Quiz (depends on Task 1)
   • Task 3: Suggest Projects (depends on Tasks 1 & 2)


In [14]:
# ================================================================================
# STEP 7: Create and Configure the Crew
# ================================================================================
# Assemble the agents into a crew with sequential task execution

education_crew = Crew(
    agents=[
        learning_material_agent,
        quiz_creator_agent,
        project_idea_agent
    ],
    tasks=[
        learning_materials_task,
        quiz_creation_task,
        project_suggestions_task
    ],
    process=Process.sequential,  # Tasks execute in order
    verbose=True
)

print("✅ Education Crew assembled!")
print(f"\n📋 Crew Configuration:")
print(f"   Process: Sequential")
print(f"   Agents: {len(education_crew.agents)}")
print(f"   Tasks: {len(education_crew.tasks)}")


✅ Education Crew assembled!

📋 Crew Configuration:
   Process: Sequential
   Agents: 3
   Tasks: 3


In [15]:
# ================================================================================
# STEP 8: Execute the Crew
# ================================================================================
# Run the crew to generate personalized educational recommendations

print("🚀 Starting Personalized Education Assistant...")
print(f"   Topics: {', '.join(user_topics)}")
print(f"   Expertise Level: {expertise_level}")
print("\n" + "="*60)
print("Executing tasks sequentially...")
print("="*60 + "\n")

# Kick off the crew execution
result = education_crew.kickoff()

print("\n" + "="*60)
print("✅ Crew execution completed!")
print("="*60)


🚀 Starting Personalized Education Assistant...
   Topics: Machine Learning, Python, Data Visualization
   Expertise Level: intermediate

Executing tasks sequentially...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5e1e58d2-60c5-44be-a296-b227b4f01651                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Curator                                                                               │
│                                                                                                                 │
│  Task: Curate a comprehensive set of learning materials for the following topics: Machine Learning, Python,     │
│  Data Visualization.                                                                                            │
│                                                                                                                 │
│  The learner's expertise level is: intermediate.                                                                │
│                                                                                                                 │
│  Your task:                                                                                                     │
│  1. Search for and select the 5 best learning resources for each topic                                          │
│  2. Include a mix of videos, articles, and practical exercises                                                  │
│  3. Ensure resources are appropriate for the intermediate level                                                 │
│  4. Create a logical learning path that builds knowledge progressively                                          │
│  5. Estimate time requirements for each resource                                                                │
│  6. List any prerequisites needed before starting                                                               │
│                                                                                                                 │
│  Focus on high-quality, up-to-date resources from reputable sources.                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: df0981db-e2bf-4b02-80b6-07156f0f1e16                                                                     │
│  Agent: Learning Material Curator                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ValidationError: 1 validation error for LearningMaterials
  Invalid JSON: expected value at line 13 column 5 [type=json_invalid, input_value='{\n  "topic": "Machine L...ding of Statistics"]\n}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

In [ ]:
# ================================================================================
# STEP 9: Display Learning Materials
# ================================================================================

print("📚 LEARNING MATERIALS")
print("="*60)

try:
    with open('learning_materials.json', 'r') as f:
        learning_data = json.load(f)
    
    print(f"\n📖 Topic: {learning_data.get('topic', 'N/A')}")
    print(f"🛤️  Learning Path: {learning_data.get('learning_path', 'N/A')}")
    print(f"⏱️  Total Time: {learning_data.get('total_estimated_time', 'N/A')}")
    
    if learning_data.get('prerequisites'):
        print(f"\n📋 Prerequisites:")
        for prereq in learning_data.get('prerequisites', []):
            print(f"   • {prereq}")
    
    print(f"\n📑 Resources ({len(learning_data.get('resources', []))}):\n")
    for i, resource in enumerate(learning_data.get('resources', []), 1):
        print(f"   {i}. {resource.get('title', 'N/A')}")
        print(f"      Type: {resource.get('resource_type', 'N/A')} | Difficulty: {resource.get('difficulty', 'N/A')}")
        print(f"      Time: {resource.get('estimated_time', 'N/A')}")
        if resource.get('url'):
            print(f"      URL: {resource.get('url')}")
        print(f"      Description: {resource.get('description', 'N/A')[:100]}...")
        print()
        
except FileNotFoundError:
    print("⚠️ Learning materials file not found. Check if the crew executed successfully.")
except json.JSONDecodeError:
    print("⚠️ Error parsing learning materials JSON.")


In [ ]:
# ================================================================================
# STEP 10: Display Quiz
# ================================================================================

print("📝 QUIZ")
print("="*60)

try:
    with open('quiz.json', 'r') as f:
        quiz_data = json.load(f)
    
    print(f"\n📋 {quiz_data.get('title', 'Quiz')}")
    print(f"📖 Topic: {quiz_data.get('topic', 'N/A')}")
    print(f"⏱️  Time Limit: {quiz_data.get('time_limit', 'N/A')}")
    print(f"✅ Passing Score: {quiz_data.get('passing_score', 'N/A')}%")
    print(f"\n📝 Instructions: {quiz_data.get('instructions', 'N/A')}")
    
    print(f"\n❓ Questions ({len(quiz_data.get('questions', []))}):\n")
    for i, q in enumerate(quiz_data.get('questions', []), 1):
        print(f"   Q{i}. {q.get('question', 'N/A')}")
        print(f"       Difficulty: {q.get('difficulty', 'N/A')}")
        for opt in q.get('options', []):
            print(f"       {opt}")
        print(f"       ✓ Correct: {q.get('correct_answer', 'N/A')}")
        print(f"       💡 Explanation: {q.get('explanation', 'N/A')}")
        print()
        
except FileNotFoundError:
    print("⚠️ Quiz file not found. Check if the crew executed successfully.")
except json.JSONDecodeError:
    print("⚠️ Error parsing quiz JSON.")


In [ ]:
# ================================================================================
# STEP 11: Display Project Suggestions
# ================================================================================

print("🛠️ PROJECT SUGGESTIONS")
print("="*60)

try:
    with open('project_suggestions.json', 'r') as f:
        project_data = json.load(f)
    
    print(f"\n👤 Expertise Level: {project_data.get('expertise_level', 'N/A')}")
    print(f"📚 Topics: {', '.join(project_data.get('topics', []))}")
    
    if project_data.get('recommended_order'):
        print(f"\n📋 Recommended Order:")
        for i, proj in enumerate(project_data.get('recommended_order', []), 1):
            print(f"   {i}. {proj}")
    
    print(f"\n💡 Projects ({len(project_data.get('projects', []))}):\n")
    for i, proj in enumerate(project_data.get('projects', []), 1):
        print(f"   {'='*55}")
        print(f"   Project {i}: {proj.get('title', 'N/A')}")
        print(f"   {'='*55}")
        print(f"   📝 Description: {proj.get('description', 'N/A')}")
        print(f"   📊 Difficulty: {proj.get('difficulty', 'N/A')}")
        print(f"   ⏱️  Duration: {proj.get('estimated_duration', 'N/A')}")
        
        if proj.get('skills_practiced'):
            print(f"   🎯 Skills: {', '.join(proj.get('skills_practiced', []))}")
        
        if proj.get('milestones'):
            print(f"   📍 Milestones:")
            for m in proj.get('milestones', []):
                print(f"      • {m}")
        
        if proj.get('resources_needed'):
            print(f"   🔧 Resources Needed:")
            for r in proj.get('resources_needed', []):
                print(f"      • {r}")
        print()
        
except FileNotFoundError:
    print("⚠️ Project suggestions file not found. Check if the crew executed successfully.")
except json.JSONDecodeError:
    print("⚠️ Error parsing project suggestions JSON.")


In [ ]:
# ================================================================================
# Display Raw Crew Output (Alternative view)
# ================================================================================

print("📄 RAW CREW OUTPUT")
print("="*60)
display(Markdown(result.raw))


---
## 🎓 Key Concepts Summary

### Core Components Used

#### 1. **Process Sequential**
Tasks are executed in a defined order, with each task potentially using the output of previous tasks.

```python
crew = Crew(
    agents=[...],
    tasks=[task1, task2, task3],
    process=Process.sequential  # Key setting
)
```

#### 2. **Pydantic Structured Outputs**
Define the exact shape of agent outputs using Pydantic models:

```python
class LearningMaterials(BaseModel):
    topic: str
    resources: List[LearningResource]

task = Task(
    ...,
    output_json=LearningMaterials,  # Structured output
    output_file="output.json"  # Saved to file
)
```

#### 3. **Custom Tools**
Create domain-specific tools by extending `BaseTool`:

```python
class ProjectSuggestionTool(BaseTool):
    name: str = "Project Suggestion Tool"
    description: str = "..."
    args_schema: Type[BaseModel] = ProjectSuggestionInput

    def _run(self, topics: str, expertise_level: str) -> str:
        # Tool logic here
        return result
```

#### 4. **Task Context**
Pass information between tasks using the `context` parameter:

```python
quiz_task = Task(
    ...,
    context=[learning_materials_task]  # Uses output from previous task
)
```

---

## 🏋️ Try It Yourself!

Experiment with the following modifications:

1. **Change Topics**: Try different subjects like "Web Development", "Data Science", "Cloud Computing"
2. **Adjust Expertise Level**: Switch between "beginner", "intermediate", and "advanced"
3. **Add More Agents**: Create a "Study Schedule Agent" that creates a weekly learning plan
4. **Enhance the Quiz**: Add different question types (true/false, fill-in-the-blank)
5. **Improve Project Tool**: Add industry-specific project templates

---

## 📚 Additional Resources

- [CrewAI Documentation](https://docs.crewai.com/)
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [CrewAI Tools](https://github.com/joaomdmoura/crewai-tools)
- [Serper API](https://serper.dev/)
